<a href="https://colab.research.google.com/github/andreasfloros/ARM-ML-Embedded/blob/main/autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount google drive to store audio data in google drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
DATASET_ROOT_DIR = '/gdrive/MyDrive/audio_data/'

In [4]:
import os
import librosa
import math
import json
import numpy as np

## Prepare and preprocess data 
Skip this step if already downloaded and preprocessed the data(data.json) on google drive

Referenced from audio_classifier pipeline

In [ ]:
import os
import requests
import tarfile


os.mkdir(DATASET_ROOT_DIR)
url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
target_path = 'audio_data/dataset.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

tar = tarfile.open(target_path, "r:gz")
tar.extractall(path='audio_data/')
tar.close()

FileExistsError: ignored

In [ ]:
def make_track_correct_size(signal, expected_num_samples_per_track):

    # print('Original track length: {}'.format(len(signal)))
    # if track is shorter than expected, append it with zeros
    if len(signal) < expected_num_samples_per_track:
      num_zeros_to_pad = expected_num_samples_per_track - len(signal)
      zeros = num_zeros_to_pad * [0.]
      extended_signal = np.append(signal, zeros)
      return extended_signal

    # if track is longer than expected, truncate it
    elif len(signal) > expected_num_samples_per_track:
      return signal[:expected_num_samples_per_track]

    # else return the original track 
    else:
      return signal

In [ ]:
def audio_track_to_features(signal, processing_method, sample_rate, window_size, window_stride, num_mfcc):
  
  if processing_method == 'fft':
    # perform Fast Fourier Transform (FFT)
    fft = np.fft.fft(signal)

    # calculate abs values on complex numbers to get magnitude
    spectrum = np.abs(fft)

    # the spectrum is symmetrical with respect to sample_rate / 2
    # so take half of the spectrum and frequency arrays
    # therefore len(half_spectrum) = sample_rate / 2
    half_spectrum = spectrum[:int(len(spectrum)/2)]

    # average every 16 samples to reduce size of array to 1 / 16 of its original size
    # e.g. sample_rate = 16k, duration = 1.024s, reduce size from 8192 to 512 
    averaged = np.mean(half_spectrum.reshape(-1, 16), axis=1)
    return averaged


  elif processing_method == 'stft':
    # perform Short Time Fourier Transform (STFT)
    stft = librosa.stft(signal = signal, 
                        n_fft = window_size, 
                        hop_length = window_stride)

    # calculate abs values on complex numbers to get magnitude
    spectrogram = np.abs(stft)

    # transpose and return the spectrogram matrix
    transposed_spectrogram = spectrogram.transpose()
    return transposed_spectrogram.flatten()


  else: # mfcc
    # perform Mel-Frequency Cepstral Coefficients (MFCC)
    mfcc = librosa.feature.mfcc(signal, 
                                sr = sample_rate, 
                                n_fft = window_size, 
                                n_mfcc = num_mfcc,
                                hop_length = window_stride)
    # transpose and return the mfcc matrix
    transposed_mfcc = mfcc.T
    return transposed_mfcc.flatten()

In [ ]:
def preprocess_entire_dataset(dataset_path, json_path, processing_method, sample_rate, expected_duration, window_size, window_stride, num_mfcc):
  # expected duration is in seconds
  expected_num_samples_per_track = int(expected_duration * sample_rate)
  
  # dictionary to later be converted to final json file
  data = {
      'mapping' : [],
      'features' : [],
      'labels' : []
  }

  # we will iterate this for each of the visited sub-directorie in order to
  # give a different label for each of them
  visited_directory_index = 0

  # iterate through all subfolders
  for dirpath, dirnames, filenames in os.walk(dataset_path):

    # # ensure we are not at the dataset root directory
    # # (os.walk provides this directory as well)
    if dirpath is not DATASET_ROOT_DIR:
    # if dirpath == 'audio_data/yes' or dirpath == 'audio_data/no':

      # obtain word labels
      dirpath_components = dirpath.split('/') # audio_data/left => ['audio_data', 'left']
      word_label = dirpath_components[-1]
      data['mapping'].append(word_label)
      print('Processing {}'.format(word_label))

      # access and process files for current word
      for f in filenames:
        
        # load audio file
        file_path = os.path.join(dirpath, f)
        signal, sample_rate = librosa.load(file_path, sr=sample_rate)

        # extend or cut signal to be equal to the expected size
        signal_correct_size = make_track_correct_size(signal, expected_num_samples_per_track)

        # obtain the features of the audio track using the function defined above
        track_features = audio_track_to_features(signal = signal_correct_size, 
                                                 processing_method = processing_method,
                                                 sample_rate = sample_rate, 
                                                 window_size = window_size, 
                                                 window_stride = window_stride, 
                                                 num_mfcc = 13)
        
        # append the audio track features to the features field of the dictionary
        data['features'].append(track_features.tolist())

        # append the current directory index as the label of this track
        data['labels'].append(visited_directory_index)
        # print('file_path: {}'.format(file_path))

      # iterate the index before visiting the next directory
      visited_directory_index = visited_directory_index + 1

  print(data['mapping'])
  print(set(data['labels']))
  # create the json file from the dictionary
  with open(json_path, 'w') as fp:
    json.dump(data, fp, indent=4)

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
!rm /gdrive/MyDrive/audio_data/_background_noise_/README.md

rm: cannot remove '/gdrive/MyDrive/audio_data/_background_noise_/README.md': No such file or directory


### Run this cell for parameters

In [9]:
JSON_PATH = DATASET_ROOT_DIR+'data.json'
PROCESSING_METHOD = 'mfcc'
SAMPLE_RATE = 16000
EXPECTED_DURATION = 1.024            # in seconds
WINDOW_SIZE_SAMPLES = 512            # in samples
WINDOW_STRIDE_SAMPLES = 320          # in samples
MFCC_COEFF_NUMBER = 13

In [ ]:
preprocess_entire_dataset(dataset_path = DATASET_ROOT_DIR, 
                   json_path = JSON_PATH, 
                   processing_method = PROCESSING_METHOD,
                   sample_rate = SAMPLE_RATE, 
                   expected_duration = EXPECTED_DURATION, 
                   window_size = WINDOW_SIZE_SAMPLES, 
                   window_stride = WINDOW_STRIDE_SAMPLES, 
                   num_mfcc = MFCC_COEFF_NUMBER)

Processing dog


KeyboardInterrupt: ignored

##Import libraries for training

In [5]:
!pip install autokeras==1.0.12

     |████████████████████████████████| 174kB 30.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=d55535b21eaec075e69160e30972aaa678a3c4b4523c2769ddf827ac9ebba660
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=3fb23ce3034f62bc35c78e791303a7326756e10d7d553e6065528eb0ec9cf0b5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Activation, BatchNormalization, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt
import autokeras as ak
import kerastuner as kt

## Load the data from data.json and prepare datasets

In [7]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    print('Loading dataset')

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data['features']).astype('float32')
    y = np.array(data['labels']).astype('int32')
    return X, y


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    print('Splitting dataset into training, validation, and test splits')

    # load data
    X, y = load_data(JSON_PATH)

    # if flatten == False:
    #   Reshape(reshape_shape, input_shape=input_shape)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    # X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets for conv networks to fit conv2D shape specs
    # currently not needed since we reshape everything anyway
    # X_train = X_train[..., np.newaxis]
    # X_validation = X_validation[..., np.newaxis]
    # X_test = X_test[..., np.newaxis]

    return X_train, X_test, y_train, y_test

In [10]:
X_train, X_test, y_train, y_test = prepare_datasets(0.25, 0.2)
print('Finished preparing training, validation, and test data')
print('X_train.shape: {}'.format(X_train.shape))

Splitting dataset into training, validation, and test splits
Loading dataset
Finished preparing training, validation, and test data
X_train.shape: (79376, 676)


In [11]:
type(X_train),type(y_train)
X_train.dtype, y_train.dtype
X_test.dtype, y_test.dtype

(dtype('float32'), dtype('int32'))

In [12]:
NUM_OF_SAMPLES = SAMPLE_RATE * EXPECTED_DURATION
NUM_OF_WINDOW_POSITIONS = math.ceil(NUM_OF_SAMPLES / WINDOW_STRIDE_SAMPLES)
NUM_OF_STFT_FREQUENCIES = int(1 + WINDOW_SIZE_SAMPLES / 2)

FFT_ARRAY_SIZE = int((NUM_OF_SAMPLES) / (2 * 16))  # 2 comes from half spectrum, 16 from averaging every 16 samples

if PROCESSING_METHOD == 'fft':
  RESHAPE_SHAPE = (1, FFT_ARRAY_SIZE, 1)
elif PROCESSING_METHOD == 'stft':
  RESHAPE_SHAPE = (NUM_OF_WINDOW_POSITIONS, NUM_OF_STFT_FREQUENCIES, 1)
else: # mfcc
  RESHAPE_SHAPE = (NUM_OF_WINDOW_POSITIONS, MFCC_COEFF_NUMBER, 1)

print('RESHAPE_SHAPE: {}'.format(RESHAPE_SHAPE))
INPUT_SHAPE = X_train.shape[1:]
print('INPUT_SHAPE:{}'.format(INPUT_SHAPE))

RESHAPE_SHAPE: (52, 13, 1)
INPUT_SHAPE:(676,)


##Build AutoKera Model




In [17]:
class SingleLayerReshapeBlock(ak.Block):
  def __init__(self, input_shape, reshape_shape):
    super().__init__()
    self.input_shape = input_shape
    self.reshape_shape = reshape_shape

  def build(self, hp, inputs=None):
      # Get the input_node from inputs.
    input_node = tf.nest.flatten(inputs)[0]
    layer = Reshape(self.reshape_shape, input_shape=self.input_shape)
    output_node = layer(input_node)
    return output_node

  # def __call__(self, *args, **kwargs):
  #   return self.build(*args, **kwargs)

def build_conv_model():
  hp = kt.HyperParameters()
  filters = hp.Choice("filters", values=[8,16,32], default=16)

  input_node = ak.Input()
  output_node = SingleLayerReshapeBlock(INPUT_SHAPE, RESHAPE_SHAPE)(input_node)
  # output_node = ak.ConvBlock(num_layers=num_layers, separable=True, max_pooling=True)(output_node)
  output_node = ak.ConvBlock(kernel_size=3,filters=filters,num_layers=1, separable=True, max_pooling=True)(output_node)
  output_node = ak.ClassificationHead()(output_node)
  auto_model = ak.AutoModel(
      inputs=input_node, outputs=output_node, objective="accuracy", tuner="greedy", overwrite=True, max_trials=100, max_model_size=100000
      )
  auto_model.fit(X_train, y_train, epochs=10)
  return auto_model

In [18]:
auto_model = build_conv_model()

Trial 39 Complete [00h 01m 45s]
accuracy: 0.5464892983436584

Best accuracy So Far: 0.7125315070152283
Total elapsed time: 00h 59m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2481/2481 [==============================] - 10s 4ms/step - loss: 2.8810 - accuracy: 0.2052
Epoch 2/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.5869 - accuracy: 0.5365
Epoch 3/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.3815 - accuracy: 0.5953
Epoch 4/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.2852 - accuracy: 0.6262
Epoch 5/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.2205 - accuracy: 0.6426
Epoch 6/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.1711 - accuracy: 0.6568
Epoch 7/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.1322 - accuracy: 0.6685
Epoch 8/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.1017 - accuracy: 0.6784
Epoch 9/10

In [19]:
print(auto_model.evaluate(X_test, y_test))

827/827 [==============================] - 2s 2ms/step - loss: 1.0365 - accuracy: 0.6975
[1.0364935398101807, 0.6974942088127136]


In [20]:
model = auto_model.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 676)]             0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 676)               0         
_________________________________________________________________
reshape (Reshape)            (None, 52, 13, 1)         0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 50, 11, 16)        41        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 5, 16)         0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 25, 5, 16)         416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 2, 16)         0     

In [21]:
model_name = 'all_words_mfcc'
model.save(model_name+'.h5')


In [22]:
# loaded_model = keras.models.load_model('all_words_mfcc.h5')
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)


# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# def representative_dataset_generator():
#   for value in X_test:
#     yield [np.array(value, dtype = np.float32, ndmin=4)]

# converter.representative_dataset = representative_dataset_generator

tflite_model = converter.convert()

# Save the model to disk
open(model_name + '.tflite', "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize(model_name + '.tflite')
print("Model is %d bytes" % basic_model_size)

INFO:tensorflow:Assets written to: /tmp/tmpklygvmdj/assets
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'
Model is 11916 bytes


In [ ]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 676)]             0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 676)               0         
_________________________________________________________________
reshape (Reshape)            (None, 52, 13, 1)         0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 50, 11, 16)        41        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 5, 16)         0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 25, 5, 16)         416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 2, 16)         0     

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the model to disk
open(model_name + '.tflite', "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize(model_name + '.tflite')
print("Model is %d bytes" % basic_model_size)

INFO:tensorflow:Assets written to: /tmp/tmpxswkq7l0/assets


INFO:tensorflow:Assets written to: /tmp/tmpxswkq7l0/assets


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


ConverterError: ignored

In [ ]:
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)


# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# def representative_dataset_generator():
#   for value in X_test:
#     yield [np.array(value, dtype = np.float32, ndmin=4)]

# converter.representative_dataset = representative_dataset_generator

tflite_model = converter.convert()

# Save the model to disk
open(model_name + '.tflite', "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize(model_name + '.tflite')
print("Model is %d bytes" % basic_model_size)

INFO:tensorflow:Assets written to: /tmp/tmp3pmrssvj/assets
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


ConverterError: ignored

In [57]:
def build_dense_model():
  hp = kt.HyperParameters()
  num_units = hp.Choice("num_units", values=[16,32,64,128,256])
  input_node = ak.Input()
  output_node = ak.DenseBlock(num_units=num_units)(input_node)
  output_node = ak.ClassificationHead(num_classes=36, loss = "categorical_crossentropy")(output_node)
  auto_model = ak.AutoModel(
      inputs=input_node, outputs=output_node, objective="accuracy", tuner="greedy", overwrite=True, max_trials=10
  )
  auto_model.fit(X_train, y_train, epochs=10)
  return auto_model

In [56]:
auto_model = build_dense_model()

AttributeError: ignored

In [ ]:
print(auto_model.evaluate(X_test, y_test))

827/827 [==============================] - 2s 2ms/step - loss: 0.8010 - accuracy: 0.7621
[0.800967276096344, 0.7621225118637085]


In [ ]:
model = auto_model.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 676)]             0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 676)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                21664     
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               16896     
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048  